## 데이터로더 생성

In [ ]:
# pip install torch torchvision torchaudio


  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
    --------------------------------------- 3.9/204.2 MB 33.7 MB/s eta 0:00:06
   -- ------------------------------------- 11.8/204.2 MB 32.1 MB/s eta 0:00:06
   --- ------------------------------------ 19.4/204.2 MB 33.1 MB/s eta 0:00:06
   ----- ---------------------------------- 27.0/204.2 MB 34.2 MB/s eta 0:00:06
   ------ --------------------------------- 34.1/204.2 MB 34.4 MB/s eta 0:00:05
   ------- -------------------------------- 39.3/204.2 MB 32.5 MB/s eta 0:00:06
   -------- ------------------------------- 44.3/204.2 MB 31.0 MB/s eta 0:00:06
   --------- ------------------------------ 48.8/204.2 MB 29.6 MB/s eta 0:00:06
   ---------- ----------------------------- 53.5/204.2 MB 28.6 MB/s eta 0:00:06
   ---------- ------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from PIL import Image


In [21]:
# 데이터 불러오기
df_train = pd.read_csv("./data/finishDF.csv")  
df_test = pd.read_csv("./data/test_dataset.csv")   

In [27]:
# 로더 정규화
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 사용모델에 맞는 크기로..
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) # 정규화화
])

In [32]:
# 데이터셋 로더화 클래스

class FaceShapeDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # 이미지 경로 가져오기
        img_path = self.dataframe.iloc[idx]['image_path']
        label = int(self.dataframe.iloc[idx]['label']) 

        # 이미지 불러오기 (OpenCV → PIL 변환)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR이므로 RGB 변환
        image = Image.fromarray(image)  # NumPy 배열을 PIL 이미지로 변환(transforms사용에 필요요)

        # 전처리 적용
        if self.transform:
            image = self.transform(image)

        return image, label

In [36]:
# 라벨을 숫자로 변환 (Label Encoding)
class_map = {label: idx for idx, label in enumerate(df_train['label'].unique())}
print("클래스 매핑:", class_map)  

df_train['label'] = df_train['label'].map(class_map)
df_test['label'] = df_test['label'].map(class_map)

클래스 매핑: {'Heart': 0, 'Oblong': 1, 'Oval': 2, 'Round': 3, 'Square': 4}


In [39]:
# 로더 생성

trainset = FaceShapeDataset(df_train, transform=transform)
testset = FaceShapeDataset(df_test, transform=transform)

# DataLoader 설정
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

In [38]:
# 확인용
images, labels = next(iter(train_loader))
print(f"이미지 텐서 크기: {images.shape}")  # (배치 크기, 채널, 높이, 너비)
print(f"라벨 샘플: {labels[:5]}")  # 앞 5개 라벨 출력

이미지 텐서 크기: torch.Size([32, 3, 224, 224])
라벨 샘플: tensor([1, 1, 1, 0, 2])


In [ ]:
# 테스트

# 모델 정의 (ResNet 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 5)  # 클래스 개수에 맞게 변경
model = model.to(device)


In [42]:
from tqdm import tqdm
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)  # 🔥 이제 정상 동작!
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 모델 저장
torch.save(model.state_dict(), "model.pth")

100%|██████████| 1242/1242 [31:00<00:00,  1.50s/it]


Epoch [1/10], Loss: 0.5076


100%|██████████| 1242/1242 [31:22<00:00,  1.52s/it]


Epoch [2/10], Loss: 0.2160


100%|██████████| 1242/1242 [31:22<00:00,  1.52s/it]


...

Epoch [8/10], Loss: 0.0486


100%|██████████| 1242/1242 [31:20<00:00,  1.51s/it]


Epoch [9/10], Loss: 0.0390


100%|██████████| 1242/1242 [31:16<00:00,  1.51s/it]

Epoch [10/10], Loss: 0.0386


In [49]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
import pandas as pd

# 📌 디바이스 설정 (GPU 가능하면 GPU 사용)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📌 모델 로드
model = models.resnet18(pretrained=False)  # 🔹 사전학습 X (새로 학습된 모델 사용)
model.fc = nn.Linear(model.fc.in_features, 5)  # 클래스 개수 맞추기
model.load_state_dict(torch.load("model.pth", map_location=device))  # 모델 불러오기
model.to(device)
model.eval()  # 평가 모드 설정

# 📌 테스트 평가 진행
correct = 0
total = 0
test_loss = 0.0
criterion = nn.CrossEntropyLoss()  # 손실 함수

with torch.no_grad():  # 테스트에서는 그래디언트 계산 X
    for images, labels in DataLoader(testset, batch_size=32, shuffle=False):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # 예측값 가져오기
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 📌 최종 결과 출력
avg_test_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total

print(f"✅ 테스트 완료: Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%")


✅ 테스트 완료: Loss: 0.4597, Accuracy: 89.35%
